# Genetic Algorithm

This notebook showcases how to use the built-in Genetic Algorithm (GA)

In [1]:
using Statistics
using EvoLP
using OrderedCollections

For this example we will use the **Rosenbrock** function:

In [2]:
@doc rosenbrock

```
rosenbrock(x; a=1, b=5)
```

**Rosenbrock** benchmark function. With $a=1$ and $b=5$, minimum is at $f([a, a^2]) = 0$

$$
f(x) = (a - x_1)^2 + b(x_2 - x_1^2)^2
$$


In a GA, we use vectors as _individuals_.

Let's start creating the population. For that, we can use a generator. Let's use the normal generator:

In [3]:
@doc normal_rand_vector_pop

```
normal_rand_vector_pop(n, μ, Σ; rng=Random.GLOBAL_RNG)
```

Generate a population of `n` vector individuals using a normal distribution with means `μ` and covariance `Σ`.

`μ` expects a vector of length *l* (i.e. length of an individual) while `Σ` expects an *l x l* matrix of covariances.

# Examples

```julia
julia> normal_rand_vector_pop(3, [0, 0], [1 0; 0 1])
3-element Vector{Vector{Float64}}:
 [-0.15290525182234904, 0.8715880371871617]
 [-1.1283800329864322, -0.9256584563613383]
 [-0.5384758126777555, -0.8141702145510666]
```


In [4]:
pop_size = 50
population = normal_rand_vector_pop(pop_size, [0, 0], [1 0; 0 1])
first(population, 3)

3-element Vector{Vector{Float64}}:
 [-0.25289759101653736, 1.0150132241600427]
 [-0.9053394512418402, 0.6058801355483802]
 [0.5784934203305488, -0.20665678122470943]

In a GA, we have _selection_, _crossover_ and _mutation_.

We can easily set up these operators using the built-ins provided by EvoLP.

Let's use rank based selection and interpolation crossover with 0.5 as the scaling factor:

In [5]:
@doc InterpolationCrossover

Interpolation crossover with scaling parameter `λ`.


In [6]:
S = RankBasedSelectionGenerational()
C = InterpolationCrossover(0.5)

InterpolationCrossover(0.5)

For mutation, we can use a Gaussian approach:

In [7]:
@doc GaussianMutation

Gaussian mutation with standard deviation `σ`, which should be a real number.


In [8]:
M = GaussianMutation(0.05)

GaussianMutation(0.05)

We can use the `Logbook` to record statistics about our run:

In [9]:
statnames = ["mean_eval", "max_f", "min_f", "median_f"]
fns = [mean, maximum, minimum, median]
thedict = LittleDict(statnames, fns)
thelogger = Logbook(thedict)

Logbook(LittleDict{AbstractString, Function, Vector{AbstractString}, Vector{Function}}("mean_eval" => Statistics.mean, "max_f" => maximum, "min_f" => minimum, "median_f" => Statistics.median), NamedTuple{(:mean_eval, :max_f, :min_f, :median_f)}[])

And now we're ready to use the `GA` built-in algorithm:

In [10]:
@doc GA

```
GA(f, pop, k_max, S, C, M)
GA(logbook::Logbook, f, population, k_max, S, C, M)
```

Generational Genetic Algorithm.

## Arguments

  * `f::Function`: objective function to **minimise**.
  * `population::AbstractVector`: a list of vector individuals.
  * `k_max::Integer`: number of iterations.
  * `S::SelectionMethod`: one of the available [`SelectionMethod`](@ref).
  * `C::CrossoverMethod`: one of the available [`CrossoverMethod`](@ref).
  * `M::MutationMethod`: one of the available [`MutationMethod`](@ref).

Returns a [`Result`](@ref).


In [11]:
result = GA(thelogger, rosenbrock, population, 500, S, C, M);

The output was suppressed so that we can analyse each part of the result separately:

In [12]:
@show optimum(result)

@show optimizer(result)

@show f_calls(result)

@show thelogger.records[end]

optimum(result) = 0.00015325530365919114
optimizer(result) = 

[0.9295343671510049, 0.9158201966396184]
f_calls(result) = 25000
thelogger.records[end] = (mean_eval = 0.07544433008393486, max_f = 0.43255087263181813, min_f = 0.00015325530365919114, median_f = 0.0424343220731829)


(mean_eval = 0.07544433008393486, max_f = 0.43255087263181813, min_f = 0.00015325530365919114, median_f = 0.0424343220731829)

In [13]:
using DataFrames

In [14]:
DataFrame(thelogger.records)

Row,mean_eval,max_f,min_f,median_f
,Float64,Float64,Float64,Float64
1,22.0251,406.9,0.447041,6.78992
2,3.61617,36.062,0.124031,1.96466
3,1.13189,3.18343,0.127583,1.07601
4,0.781777,1.6644,0.309661,0.711803
5,0.593735,0.935043,0.294026,0.588684
6,0.527621,0.766033,0.315916,0.518089
7,0.522381,0.745129,0.37027,0.527158
8,0.493569,0.807639,0.275269,0.498038
9,0.429047,0.598718,0.254738,0.43252
